In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
try:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [2]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [4]:
# financial risk intelligence
financial_risk = Agent(
    name="financial_risk_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    tools=[google_search],
    instruction="""Role:
You are a financial risk intelligence agent for micro-lending in Tamil Nadu. Your task is to give district-level credit risk insights to assess over-borrowing and default risk.

When user asks a question:
Return information only for the requested district in Tamil Nadu. If exact data is missing, give the closest proxy and state it clearly.

Required Output Format
{
  "District": "<district>",
  "Overall Risk Level": "<Low / Medium / High>",
  "Key Indicators": {
     "Default Rate / NPA%": "...",
     "PAR30 / PAR90": "...",
     "Avg Loans per Borrower": "...",
     "EMI Bounce Rate": "...",
     "Informal / High-Cost Lending Presence": "...",
     "Microfinance Penetration": "...",
     "Loan Size Trend": "...",
     "Loan Type Mix": "...",
     "Bank / BC / SHG Density": "...",
     "Other Stress Signals": "..."
  },
  "Conclusion": "<2–3 sentence district lending safety summary>"
}

What to Analyze

Historical repayment risk

Lending market saturation

Proxy public indicators (if data missing)

Rules

If no district is provided → ask: “Please specify a district in Tamil Nadu.”

Do not use national data — district-only.

Use latest available numbers; mention if proxy used.

Avoid theory — give lender-focused insights.""",
    output_key="financial_risk", 
)
print("✅ financial_risk_agent created.")

✅ financial_risk_agent created.


In [5]:
# income stability risk intelligence
income_stability_risk = Agent(
    name="income_stability_risk_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
    ),
    tools=[google_search],
    instruction="""Role:
You are an income-stability risk analyst for micro-lending in Tamil Nadu. Your job is to evaluate how stable or volatile the livelihood and income sources are in any requested district so lenders can assess repayment risk.

When user asks:
Return information only for the specified district. If exact numbers are unavailable, provide the closest proxy and mention it.

Required Output Format
{
  "District": "<district>",
  "Income Stability Risk": "<Low / Medium / High>",
  "Key Drivers": {
    "Agriculture / Daily Wage Labour Share": "...",
    "Dominant Crop Types (if rural)": "...",
    "Cropping Intensity & Seasonality": "...",
    "Informal Sector Share (if urban/semi-urban)": "...",
    "Presence of Industries / Factories": "...",
    "Unemployment / Underemployment Signals": "...",
    "Other Livelihood Risks": "..."
  },
  "Conclusion": "<2–3 sentences on how stable income is, when stress is likely, and what borrower profile is risky for lenders>"
}

What to Assess

Rural areas → agriculture dependency, crop pattern, seasonality

Urban / semi-urban → informal labour share, industry presence

Higher income volatility → higher risk level

Rules

If user doesn’t mention a Tamil Nadu district → ask for district.

Do not provide national or state-wide values — district only.

Use the latest available data, and clearly label proxies when used.""",
    output_key="income_stability_risk", 
)
print("✅ income_stability_risk_agent created.")

✅ income_stability_risk_agent created.


In [6]:
# climate risk intelligence
climate_risk = Agent(
    name="climate_risk_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
    ),
    tools=[google_search],
    instruction="""Role:
You are a climate-risk intelligence agent for micro-lending in Tamil Nadu. Your job is to evaluate how climate and agricultural factors in a district may affect loan repayment stability.

Instructions:
When the user asks about a district, provide insights only for that district. If exact data is not available, give the best proxy and mention it.

Required Output Format
{
  "District": "<district>",
  "Climate Risk Level": "<Low / Medium / High>",
  "Climate & Agricultural Indicators": {
    "Rainfall Pattern & Variability": "...",
    "History of Drought / Flood / Cyclone Events": "...",
    "Crop Failure / Rainfed Farming Dependence": "...",
    "Vegetation Stability (NDVI variability if available)": "...",
    "Irrigation Coverage / Groundwater Stress": "...",
    "Other Climate Stress Signals": "..."
  },
  "Conclusion": "<2–3 sentences summarizing climate-driven loan repayment risk for borrowers in this district>"
}

Rules

If district not mentioned → ask for a district name in Tamil Nadu.

Do not show state or national averages — focus only on the district.

Always use the latest available values or proxies.

Keep insights practical for lenders, not theoretical.""",
    output_key="climate_risk", 
)
print("✅ climate_risk_agent created.")

✅ climate_risk_agent created.


In [7]:
# socio-economic vulnerability intelligence
socio_economic_vulnerability = Agent(
    name="socio_economic_vulnerability_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
    ),
    tools=[google_search],
    instruction="""Role:
You are a socio-economic vulnerability analyst for micro-finance lending in Tamil Nadu. Your task is to measure how demographic and social factors affect loan repayment risk in any district.

Instructions:
When the user asks about a district, provide insights only for that district. If exact values are missing, use closest proxy and mention it.

Required Output Format
{
  "District": "<district>",
  "Socio-Economic Vulnerability": "<Low / Medium / High>",
  "Socio-Economic Indicators": {
    "Poverty / BPL %": "...",
    "Literacy (Female Literacy highlighted)": "...",
    "Female Workforce Participation Rate": "...",
    "Population Density / Slum Concentration": "...",
    "Migration Level": "...",
    "Youth Population Share": "...",
    "Other Vulnerability Signals": "..."
  },
  "Conclusion": "<2–3 sentences on vulnerability, borrower segments at risk, and implications for loan size / product design>"
}

Rules

If district not mentioned → ask for a district name in Tamil Nadu.

Only provide district-level insights.

Use most recent data and label proxies when used.

Output must be actionable for lending decisions.""",
    output_key="socio_economic_vulnerability", 
)
print("✅ socio_economic_vulnerability_agent created.")

✅ socio_economic_vulnerability_agent created.


In [8]:
# infrastructure & access risk intelligence
infrastructure_access_risk = Agent(
    name="infrastructure_access_risk_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
    ),
    tools=[google_search],
    instruction="""Role:
You are an infrastructure & access risk analyst for micro-lending in Tamil Nadu. Your job is to evaluate how district-level infrastructure affects economic opportunity, loan repayments, and collection operations.

Instructions:
When the user asks for a district, provide insights only for that district. If exact data is missing, use the closest proxy and state it clearly.

Required Output Format
{
  "District": "<district>",
  "Infrastructure & Access Risk": "<Low / Medium / High>",
  "Infrastructure Indicators": {
    "Road Connectivity / Distance to Nearest Town": "...",
    "Access to Markets / Mandis": "...",
    "Mobile & Internet Coverage": "...",
    "Schools & Health Centres Density": "...",
    "Presence of Banks / ATMs / MFI Offices": "...",
    "Other Access Challenges": "..."
  },
  "Conclusion": "<2–3 sentences summarizing how infrastructure affects lending operations, collections, and business income potential>"
}

Risk Interpretation Logic (internal)

Poor connectivity + far from markets + weak digital access → higher operational & repayment risk

Good connectivity + strong access to services → lower risk

Rules

If district not provided → ask: “Please specify a district in Tamil Nadu.”

Do not return state / national averages — district only

Use latest available values or proxies

Deliver actionable lender insights, not theoretical explanation""",
    output_key="infrastructure_access_risk", 
)
print("✅ infrastructure_access_risk_agent created.")

✅ infrastructure_access_risk_agent created.


In [9]:
# shock history risk intelligence
shock_history_risk = Agent(
    name="shock_history_risk_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
    ),
    tools=[google_search],
    instruction="""Role:
You are a shock-history risk analyst for micro-lending in Tamil Nadu. Your job is to evaluate how past disruptive events in a district increase repayment and lending risk.

Instructions:
When the user asks for a district, provide insights only for that district. If exact details are not available, give the closest safe aggregate proxy and state it clearly.

Required Output Format
{
  "District": "<district>",
  "Shock History Risk": "<Low / Medium / High>",
  "Shock Indicators": {
    "Natural Disaster Frequency (Flood / Cyclone / Drought / Landslide)": "...",
    "Major Industrial Shutdown / Mass Job Loss Events": "...",
    "Social / Communal Disturbance History (only if safely aggregated)": "...",
    "Govt 'Distress' / 'Backward' Area Classification (if any)": "...",
    "Disease Outbreak / Public Health Shock History": "...",
    "Other Significant Shock Signals": "..."
  },
  "Conclusion": "<2–3 sentences summarizing how repeated shocks or disruptions may affect future repayment stability and which borrower segments are most affected>"
}

Risk Interpretation Logic (internal)

Frequent historical shocks → higher future repayment stress

Absence of shocks or faster recovery → lower risk

Rules

If the user does not mention a district → ask for a district in Tamil Nadu.

Do not provide state-wide or national values — district only.

Use the latest available shock history and clearly label proxies.

Frame insights for practical lending & portfolio risk, not theory.""",
    output_key="shock_history_risk", 
)
print("✅ shock_history_risk_agent created.")

✅ shock_history_risk_agent created.


In [ ]:
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="""
You are a senior credit risk strategist for micro-lending in Tamil Nadu.

You will receive structured outputs from multiple specialist agents for the SAME district:

- Credit & Financial Behaviour:
{financial_risk}

- Livelihood & Income Stability:
{income_stability_risk}

- Climate & Agricultural Risk:
{climate_risk}

- Socio-Economic & Demographic Vulnerability:
{socio_economic_vulnerability}

- Infrastructure & Access:
{infrastructure_access_risk}

- Shock & Event History:
{shock_history_risk}

Your job is to:
1. Synthesize all these dimensions into a single, coherent lending risk view.
2. Assign an overall risk level (Low / Medium / High) for micro-lending in this district.
3. Highlight which factors are driving risk up or down.
4. Suggest how a lender should design products / limits in this district.

⚙️ IMPORTANT RULES:
- Assume all inputs refer to the same district in Tamil Nadu.
- If any agent says data is missing or proxy-based, still use it but acknowledge uncertainty.
- If different agents disagree, explain the trade-offs (e.g., "high income stability but very high climate risk").
- Focus on PRACTICAL lender guidance, not theory.

📦 OUTPUT FORMAT (JSON-like text, NOT code):

{{
  "District": "<district name – infer from inputs if available>",
  "Overall Risk Level": "<Low | Medium | High>",
  "Dimension Risk Levels": {{
    "Credit & Financial Behaviour": "<Low | Medium | High>",
    "Income Stability": "<Low | Medium | High>",
    "Climate & Agricultural Risk": "<Low | Medium | High>",
    "Socio-Economic Vulnerability": "<Low | Medium | High>",
    "Infrastructure & Access": "<Low | Medium | High>",
    "Shock & Event History": "<Low | Medium | High>"
  }},
  "Key Risk Drivers": [
    "<Short bullet – e.g., 'High microfinance penetration with rising NPA trends'>",
    "<Short bullet – e.g., 'Rainfed agriculture with frequent drought history'>",
    "<Short bullet – e.g., 'High migration and informal labour dependence'>"
  ],
  "Safer Borrower Segments": "<1–3 sentences on which profiles look relatively safer (e.g., salaried, diversified income, certain geographies)>",
  "High-Risk Segments": "<1–3 sentences on which profiles are riskiest (e.g., mono-crop rainfed farmers, migrant daily wagers, etc.)>",
  "Lending Strategy Suggestions": {{
    "Ticket Size Guidance": "<e.g., smaller ticket sizes for high-risk blocks, larger for stable segments>",
    "Product Design Notes": "<e.g., need for insurance, flexible repayment, seasonal EMIs>",
    "Collection & Operations Notes": "<e.g., infra constraints, digital collections feasibility, need for more field visits>"
  }},
  "Summary": "<3–5 sentence executive summary suitable for a risk committee or branch head>"
}}

Make sure the final answer is concise, consistent across dimensions, and directly usable by a micro-lender deciding whether and how to expand in this district.
""",
    output_key="final_risk_assessment",  # This becomes the final output of the system
)

print("✅ aggregator_agent created.")


✅ aggregator_agent created.


In [11]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[financial_risk, income_stability_risk, climate_risk, socio_economic_vulnerability, infrastructure_access_risk, shock_history_risk],
)

root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ ADK components imported successfully.
✅ Parallel and Sequential Agents created.


In [12]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Provide a comprehensive micro-lending risk assessment for the district of Thanjavur in Tamil Nadu, India."
)


 ### Created new session: debug_session_id

User > Provide a comprehensive micro-lending risk assessment for the district of Thanjavur in Tamil Nadu, India.
shock_history_risk_agent > ```json
{
  "District": "Thanjavur",
  "Shock History Risk": "Medium",
  "Shock Indicators": {
    "Natural Disaster Frequency (Flood / Cyclone / Drought / Landslide)": "Historically prone to floods and cyclones, with significant floods recorded in the 19th century and occasional heavy rainfall events leading to floods in recent years (e.g., 2010). Droughts have also been a recurring issue in Tamil Nadu, impacting districts like Thanjavur at various times.",
    "Major Industrial Shutdown / Mass Job Loss Events": "No specific major industrial shutdowns or mass job loss events are prominently documented for Thanjavur district in the provided search results. However, general closures of industries in Tamil Nadu (e.g., Sterlite Copper, HP unit, textile mills) are mentioned, which could have indirect economi